# **Introducción**

En este cuaderno de Colab, se ejecuta una operación de entrenamiento de una neurona que compone una red neuronal de un modelo de Machine Learning

El entrenamiento consiste en una capa con *n* neuronas y *n* pesos que la relacionan con una neurona de la segunda capa.

El valor de la neurona de la capa 2 se determina por la funcion sigmoide de la suma pesada de las neuronas de la capa 1, como se define en la siguiente función: **σ(w1 a1 + w2 a2 + ... + wn an)**

La función sigmoide devuelve un valor entre 0 y 1

# **Armado del ambiente**
No es necesario ningun proceso anterior para el armado del ambiente

# **Desarrollo CPU**

In [191]:
from datetime import datetime
import numpy as np
import math

tiempo_total = datetime.now()
np.random.seed(int(tiempo_total.timestamp()))

cant_neuronas = 1000000#@param {type:"number"}

# Definición de función que transforma el tiempo en  milisegundos 
def tiempo_en_ms(dt):
  return (dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#los valores de las neuronas deben ser entre 0 y 1, por eso les resto la parte entera
def reduce_weights(x):
  return x - int(x)

#funcion sigmoide, devuelve un valor real entre 0 y 1
def sigmoid(x):
    if x > 10000:
      return 1
    elif x < -10000:
      return 0
    else:
      return 1 / (1 + math.exp(-x))

# los valores de las neuronas deben estar entre 0 y 1
vec_neuronas = np.random.rand(cant_neuronas)
vec_neuronas = vec_neuronas.astype(np.float32())
vec_neuronas = np.vectorize(reduce_weights)(vec_neuronas)

# los valores de los pesos pueden tomar cualquier valor real
vec_pesos = np.random.randn(cant_neuronas)
vec_pesos = vec_pesos.astype(np.float32())

# # valores dummy para corroborar con el ejercicio en cpu
# vec_neuronas = np.array([0.66, 0.8,  0.24, 0.01, 0.56, 0.47, 0.29, 0.94, 0.86, 0.9 ])
# vec_pesos = np.array([ 0.81,  1.41, -1.38,  0.63, -0.36, 0.2,  -0.87, -0.31, -0.18,  0.43])

tiempo_ciclo = datetime.now()

acum = 0
for i in range(cant_neuronas):
  acum += vec_neuronas[i] * vec_pesos[i]

valor_neurona_layer_2 = sigmoid(acum)

tiempo_ciclo = datetime.now() - tiempo_ciclo
tiempo_total = datetime.now() - tiempo_total

print('Valor de la neurona: {:.6f}'.format(valor_neurona_layer_2))
print('Tiempo ciclo: {:.2f} ms'.format(tiempo_en_ms(tiempo_ciclo)))
print('Tiempo total: {:.2f} ms'.format(tiempo_en_ms(tiempo_total)))

Valor de la neurona: 0.000000
Tiempo ciclo: 554.76 ms
Tiempo total: 859.73 ms


# **Desarrollo GPU**

In [ ]:
!pip install pycuda

In [190]:
from datetime import datetime
import numpy as np
import math

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

tiempo_total = datetime.now()
np.random.seed(int(tiempo_total.timestamp()))

cant_neuronas = 1000000#@param {type:"number"}

# Definición de función que transforma el tiempo en  milisegundos 
def tiempo_en_ms(dt):
  # return 5
  return (dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#los valores de las neuronas deben ser entre 0 y 1, por eso les resto la parte entera
def reduce_weights(x):
  return x - int(x)

#funcion sigmoide, devuelve valor entre 0 y 1
def sigmoid(x):
    if x > 10000:
      return 1
    elif x < -10000:
      return 0
    else:
      return 1 / (1 + math.exp(-x))

# los valores de las neuronas deben estar entre 0 y 1
vec_neuronas = np.random.rand(cant_neuronas)
vec_neuronas = np.vectorize(reduce_weights)(vec_neuronas)

# los valores de los pesos pueden tomar cualquier valor real
vec_pesos = np.random.randn(cant_neuronas)

# # valores dummy para corroborar con el ejercicio en cpu
# vec_neuronas = np.array([0.66, 0.8,  0.24, 0.01, 0.56, 0.47, 0.29, 0.94, 0.86, 0.9 ])
# vec_pesos = np.array([ 0.81,  1.41, -1.38,  0.63, -0.36, 0.2,  -0.87, -0.31, -0.18,  0.43])

# vector auxiliar para guardar los valores
vec_aux = np.empty_like(vec_neuronas)
vec_aux = vec_aux.astype(np.float64())

#reservo la memoria para los vectores
vec_neuronas_gpu = cuda.mem_alloc(vec_neuronas.nbytes)
vec_pesos_gpu = cuda.mem_alloc(vec_pesos.nbytes)
vec_aux_gpu = cuda.mem_alloc(vec_aux.nbytes)

#copio la memoria a la GPU
cuda.memcpy_htod(vec_neuronas_gpu, vec_neuronas)
cuda.memcpy_htod(vec_pesos_gpu, vec_pesos)
cuda.memcpy_htod(vec_aux_gpu, vec_aux)

kernel_acum = module.get_function("kernel_acum")

dim_hilo = 256
dim_bloque = np.int( (cant_neuronas+dim_hilo_x-1) / dim_hilo_x )

tiempo_ciclo = datetime.now()

module = SourceModule("""
__global__ void kernel_acum(int cant_neuronas, double* vec_neuronas,
double* vec_pesos, double* aux)
{
  //Coordenadas de los threads
  int idx = threadIdx.x + blockIdx.x*blockDim.x;

  //Verifico que esten dentro de los valores
  if(idx < cant_neuronas)
  {
    aux[idx] = vec_neuronas[idx] * vec_pesos[idx];
  }
}

""")


kernel_acum(np.int32(cant_neuronas), vec_neuronas_gpu, vec_pesos_gpu, vec_aux_gpu,
            block=(dim_hilo, 1, 1), grid=(dim_bloque, 1, 1))

cuda.memcpy_dtoh(vec_aux, vec_aux_gpu)
cuda.memcpy_dtoh(vec_neuronas, vec_neuronas_gpu)
cuda.memcpy_dtoh(vec_pesos, vec_pesos_gpu)

# uso un vector auxiliar para la suma porque no encontre forma de usar un acumulador con memoria compartida dentro de CUDA
acum = np.sum(vec_aux) 

tiempo_ciclo = datetime.now() - tiempo_ciclo
tiempo_total = datetime.now() - tiempo_total

valor_neurona_layer_2 = sigmoid(acum)

print('Valor de la neurona: {:.6f}'.format(valor_neurona_layer_2))
print('Tiempo ciclo: {:.2f} ms'.format(tiempo_en_ms(tiempo_ciclo)))
print('Tiempo total: {:.2f} ms'.format(tiempo_en_ms(tiempo_total)))


Valor de la neurona: 1.000000
Tiempo ciclo: 6.55 ms
Tiempo total: 318.26 ms


# **Bibliografía**

"Que es una red neuronal? | Aprendizaje Profundo, capítulo 1" https://www.youtube.com/watch?v=aircAruvnKk&ab_channel=3Blue1Brown